<a href="https://colab.research.google.com/github/Gail529/Recommendation-system/blob/master/emotion_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
from tqdm import tqdm_notebook as tqdm 
from tqdm import trange
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
data=pd.read_csv('/content/rock_songs.csv') #dataframe containing songs

In [ ]:
emo_lex=pd.read_excel('/content/NRC-Emotion-Lexicon-v0.92-In105Languages-Nov2017Translations.xlsx') # emotion lexicon 

In [ ]:
emolex_df=emo_lex[['English (en)','Positive','Negative','Anger','Anticipation','Disgust','Fear','Joy','Sadness','Surprise','Trust']]
emotions=emolex_df.columns.drop('English (en)')
emolex_df.rename(columns={'English (en)':'word'},inplace=True)

In [ ]:
import re
import string
for i in range(180):
    song=data.iloc[i,2]
    lyrics=re.sub('['+string.punctuation+']','',song)# removing punctuations
    data.iat[i,2]=lyrics


In [ ]:
emolex_df.head(5)

In [ ]:
data.head()

In [ ]:
#the function passes the column(col) of the respective dataframe passed as arguements
def text_emotion(song_df,col):
  new_song_df=song_df.copy()
  #new  dataframe that will store the emotion scores
  emo_df=pd.DataFrame(0,index=new_song_df.index, columns=emotions)
  lemmatizer=WordNetLemmatizer()

  
  with tqdm(total=len(list(new_song_df.iterrows()))) as pbar:
    for i ,row in new_song_df.iterrows(): 
        pbar.update(1) #update the progress bar
        song=word_tokenize(new_song_df.loc[i][col]) #the body of text for each individual song(row)

        for word in song:
            new=lemmatizer.lemmatize(word)#lemmatizing the words
            emo_score=emolex_df[emolex_df.word == new]     
            if not emo_score.empty:
                for emotion in list(emotions):
                    emo_df.at[i,emotion] += emo_score[emotion]


    new_song_df=pd.concat([new_song_df,emo_df] ,axis=1)
    return new_song_df

In [ ]:
final_df=text_emotion(data,'1')

In [ ]:
final_df.head(10)

,Unnamed: 0,song-title,lyrics,Positive,Negative,Anger,Anticipation,Disgust,Fear,Joy,Sadness,Surprise,Trust
0,0,Yellow,look stars look shine everything yeah yellow c...,14,0,0,2,0,0,6,0,0,2
1,1,Viva La Vida,used rule world seas would rise gave word morn...,27,10,6,5,5,8,9,6,3,17
2,2,In My Place,place place lines could nt change lost oh yeah...,8,13,0,20,0,2,7,11,1,8
3,3,God Put A Smile Upon Your Face,go nobody knows got ta say way god give style ...,24,4,1,15,1,7,18,3,15,18
4,4,Princess of China,oh upon time somebody ran somebody ran away sa...,6,12,9,7,0,9,2,11,0,2
5,5,Up in Flames,time know gone salt water tasted long know wro...,1,1,0,4,0,1,0,0,0,0
6,6,Charlie Brown,stole key took car downtown lost boys meet too...,6,8,2,0,3,6,0,5,0,0
7,7,Paradise,girl expected world flew away reach ran away s...,3,4,1,5,1,3,2,1,3,2
8,8,Us Against the World,oh morning come bursting clouds amen lift blin...,7,7,6,8,2,7,6,7,4,6
9,9,Clocks,lights go ca nt saved tides tried swim brought...,3,6,4,3,1,5,1,4,0,0


In [ ]:
cols=final_df.columns
col=cols[0]
final_df.rename(columns={'0':'song-title','1':'lyrics'},inplace=True)


In [ ]:
final_df.to_csv('rock_emotion_scores.csv')